survival	Survival	0 = No, 1 = Yes
pclass	Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd
sex	Sex	
Age	Age in years	
sibsp	# of siblings / spouses aboard the Titanic	
parch	# of parents / children aboard the Titanic	
ticket	Ticket number	
fare	Passenger fare	
cabin	Cabin number	
embarked	Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton

In [3]:
import pandas as pd
import seaborn as sns
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [117]:
raw = pd.read_csv('train.csv')
test = pd.read_csv('test.csv', index_col='PassengerId')
raw.columns = pd.Series(raw.columns).apply(lambda r: r.lower())
raw.sex = raw.sex.map({'female': 0, 'male': 1})
raw.head(4)

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S


In [231]:
test.head(3)

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


In [217]:
L=[900,1,1,'lenka-pechenka',0,29,0,0,'rrr',70,'tt','S']
pd.DataFrame(L).T

,0,1,2,3,4,5,6,7,8,9,10,11
0,900,1,1,lenka-pechenka,0,29,0,0,rrr,70,tt,S


In [118]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
passengerid    891 non-null int64
survived       891 non-null int64
pclass         891 non-null int64
name           891 non-null object
sex            891 non-null int64
age            714 non-null float64
sibsp          891 non-null int64
parch          891 non-null int64
ticket         891 non-null object
fare           891 non-null float64
cabin          204 non-null object
embarked       889 non-null object
dtypes: float64(2), int64(6), object(4)
memory usage: 83.6+ KB


In [119]:
raw.age=raw.age.fillna(raw.age.mean())
raw.cabin = raw.cabin.fillna('na')
raw.embarked= raw.embarked.fillna('na')

In [229]:
f1=raw[raw.survived==1].groupby(['pclass','age'])['passengerid'].count()
f2=raw[raw.survived==0].groupby(['age'])['passengerid'].count()
pd.DataFrame(f1)


#plt.bar(f2.index,f2,color='red')
#plt.bar(f1.index,f1,color='green')

passengerid
pclass age                   
1      0.920000             1
       4.000000             1
       11.000000            1
       14.000000            1
       15.000000            1
       16.000000            3
       17.000000            3
       18.000000            3
       19.000000            3
       21.000000            2
       22.000000            4
       23.000000            3
       24.000000            5
       25.000000            2
       26.000000            2
       27.000000            3
       28.000000            2
       29.000000            1
       29.699118           14
       30.000000            5
       31.000000            3
       32.000000            2
       33.000000            3
       34.000000            1
       35.000000            9
       36.000000            7
       37.000000            1
       38.000000            4
       39.000000            4
       40.000000            3
...                       ...
3      9.000000             2
       12.000000            1
       13.000000            1
       14.000000            1
       15.000000            3
       16.000000            3
       17.000000            1
       18.000000            4
       19.000000            3
       20.000000            3
       21.000000            2
       22.000000            5
       23.000000            1
       24.000000            4
       25.000000            2
       26.000000            4
       27.000000            6
       29.000000            4
       29.699118           34
       30.000000            2
       31.000000            3
       32.000000            5
       33.000000            1
       35.000000            1
       36.000000            1
       38.000000            1
       39.000000            1
       44.000000            1
       45.000000            1
       63.000000            1

[128 rows x 1 columns]

In [123]:
raw.columns

Index([u'passengerid', u'survived', u'pclass', u'name', u'sex', u'age',
       u'sibsp', u'parch', u'ticket', u'fare', u'cabin', u'embarked'],
      dtype='object')

In [168]:
num_data = raw[['pclass','sex', 'age','sibsp', 'parch','fare']]
cat_data = raw[['cabin','embarked']].drop(['cabin'],axis=1)
y=raw['survived']
cat_data.head(2)

,embarked
0,S
1,C


In [156]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer as DV
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [218]:
L

[900, 1, 1, 'lenka-pechenka', 0, 29, 0, 0, 'rrr', 70, 'tt', 'S']

In [180]:
scaler = StandardScaler()
X_scaled=scaler.fit_transform(num_data)
#pd.DataFrame(X)
encoder = DV(sparse = False)
X_cat= encoder.fit_transform(cat_data.T.to_dict().values())
X=np.hstack([X_scaled,X_cat])
pd.DataFrame(X).head(2)
XL=L



,0,1,2,3,4,5,6,7,8,9
0,0.827377,0.737695,-0.592481,0.432793,-0.473674,-0.502445,0.0,0.0,1.0,0.0
1,-1.566107,-1.355574,0.638789,0.432793,-0.473674,0.786845,1.0,0.0,0.0,0.0


In [222]:
(X_train,
X_test,
y_train,
y_test)=train_test_split(X,y,test_size=0.3,stratify=y)

xgb = XGBClassifier()
xgb.fit(X_train,y_train)

LG = LogisticRegression(class_weight='balanced',penalty='l1')
LG.fit(X_train,y_train)

print ' LG ' +str(roc_auc_score(y_test,LG.predict(X_test)))
print ' XGB ' +str(roc_auc_score(y_test,xgb.predict(X_test)))

 LG 0.794410120624
 XGB 0.797999411592


0.8429538099441013 - Scaler + cat_dat(embar)

In [228]:
print raw.columns
pd.DataFrame(LG.coef_)


Index([u'passengerid', u'survived', u'pclass', u'name', u'sex', u'age',
       u'sibsp', u'parch', u'ticket', u'fare', u'cabin', u'embarked'],
      dtype='object')


,0,1,2,3,4,5,6,7,8,9
0,-0.856971,-1.177105,-0.543908,-0.324722,-0.04881,0.160789,0.183599,0.0,-0.318912,0.0
